# Features analysis tutorial

This tutorial shows how to perform an exploratory analysis of the features data: to visualize features distribution in classes, plot the feature correlation matrix, check Mann-Whitney U-test p-values, plot univariate ROC (and calculate AUC) for each feature, perform volumetric analysis, and save all the scores. We will use PyRadiomics features as variables and binary survival label as an outcome.  
Then we will clean the data and perform the basic radiomics pipeline.

Importing modules:

In [ ]:
import os,sys
from pmtool.AnalysisBox import AnalysisBox
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression

## Required data

To run the tutorial, you will need the following files, available in the 'data/features' folder of the *precision-medicine-toolbox* repository:  
- '*extended_clinical_df.xlsx*' - this is the clinical data file provided with the Lung1 dataset (Aerts et al., 2014); we added binary variables of 1-, 1.5-, and 2-years survival, based on the presented data,  
- '*extracted_features_full.xlsx*' - this is a file with the PyRadiomic features values (can be extracted in the imaging module tutorial) for **all** the patients from Lung1 dataset.

## Exploratory analysis

If you have 2 classes in your dataset, the following functionality is availabe for your analysis. We will divide Lung1 dataset by 1 year survival label, therefore having 2 classes.

Set up the parameters to get the data:

In [ ]:
parameters = {
    'feature_path': "extracted_features_full.xlsx", # path to csv/xls file with features
    'outcome_path': "extended_clinical_df.xlsx", #path to csv/xls file with outcome
    'patient_column': 'Patient', # name of column with patient ID
    'patient_in_outcome_column': 'PatientID', # name of column with patient ID in clinical data file
    'outcome_column': '1yearsurvival' # name of outcome column
}

Initialise the feature set (you will see a short summary):

In [ ]:
fs = AnalysisBox(**parameters)

Print some attributes of the feature set - first 10 patient IDs and first 10 feature names:

In [ ]:
print ('Patient IDs: ', fs._patient_name[:10])
print ('Total patients: ', len(fs._patient_name))
print ('\nFeature names: ', fs._feature_column[:10])
print ('Total features: ', len(fs._feature_column))

Print the head of the composed dataframe, containing both the variables and the outcome:

In [ ]:
fs._feature_outcome_dataframe.head(5)

Visualize feature values distribution in classes (will pop up in an interactive .html report):

In [ ]:
fs.plot_distribution(fs._feature_column)

Visualize mutual feature correlation coefficient (Spearman's) matrix (in .html report):

In [ ]:
fs.plot_correlation_matrix(fs._feature_column)

Visualize Mann-Whitney (Bonferroni corrected) p-values for binary classes test (in .html report):

In [ ]:
fs.plot_MW_p(fs._feature_column)

Vizualize univariate ROC curves:

In [ ]:
fs.plot_univariate_roc(fs._feature_column, auc_threshold=0.70)

Calculate the basic statistics for each feature: number of NaN, mean, std, min, max; if applicable: MW-p, univariate ROC AUC, volume correlation:

In [ ]:
fs.calculate_basic_stats(volume_feature='original_shape_VoxelVolume')

Check the excel table:

In [ ]:
print('Basic statistics for each feature')
pd.read_excel('extracted_features_full_basic_stats.xlsx')

Volume analysis will show you if your features have a high correlation to volume and if volume itself is a predictive feature in separation of 2 classes. You need to have a volume feature in your dataset and send it as a function parameter (in our case it is *'original_shape_VoxelVolume'*). 

In [ ]:
fs.volume_analysis(volume_feature='original_shape_VoxelVolume')

## Outliers detection

Now we are preparing the data, which is ouside of the toolbox functionality, therefore, we will present the features and outcomes in separate dataframes,

In [ ]:
data_all = fs._feature_dataframe.copy().drop('ROI', axis=1)
outcome_all = fs._outcome

We already know that we have missing values, so we will perform imputation with mean feature values to check the dataset for outliers:

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(data_all)
x_imp = imp.transform(data_all)

Rescaling the values for PCA:

In [ ]:
x_stand_scaled = StandardScaler().fit_transform(x_imp)

Performing PCA transformation to present every observation in 2D;

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x_stand_scaled)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

Plotting the observations in 2D space:

In [ ]:
plt.scatter(principalDf['principal component 1'], principalDf['principal component 2'])
plt.xlabel('principal component 1')
plt.ylabel('principal component 2')
plt.title('2 component PCA')
plt.show()

performing the same steps for 1-component decomposition:

In [ ]:
pca_1D = PCA(n_components=1)
principalComponents_1D = pca_1D.fit_transform(x_stand_scaled)

plt.hist(principalComponents_1D, bins = 150)
plt.xlabel('principal component 1')
plt.title('1 component PCA')
plt.show()

Fixing some discovered findings:

In [ ]:
data_all.drop(index=['LUNG1-014_000000_GTV-1_mask'], inplace=True)
outcome_all.drop(index=['LUNG1-014_000000_GTV-1_mask'], inplace=True)

data_all.shape

Visualizing feature distributions ib borplots (quartile outliers detection):

In [ ]:
for i in range (0, len(fs._feature_column)-1):
    
    ax = sns.boxplot(data=data_all[fs._feature_column[i]], orient="h")
    plt.title(fs._feature_column[i])
    plt.show()

The other approach for outliers detection - based on Z-scoring:

In [ ]:
def z_score_method(df, variable_name):
    
    columns = df.columns
    z = np.abs(stats.zscore(df))
    threshold = 3
    outlier = []
    index=0
    for item in range(len(columns)):
        if columns[item] == variable_name:
            index = item
    for i, v in enumerate(z[:, index]):
        if v > threshold:
            outlier.append(i)
        else:
            continue
    return outlier

Let's identify indices of the patients, who have outliers for every feature value:

In [ ]:
for i in range (0, len(fs._feature_column)-1):
    
    outlier_z = z_score_method(data_all, fs._feature_column[i])
    print(fs._feature_column[i], outlier_z)

Dropping a feature with duplicated value:

In [ ]:
data_all.drop('original_gldm_LargeDependenceHighGrayLevelEmphasis', axis=1, inplace=True)

data_all.shape

## Feature imputation

Convertion DataFrame into array (input for the Python package):

In [ ]:
X = np.array(data_all)
y = np.array(outcome_all)

print (X.shape, y.shape)

Imputation with mean:

In [ ]:
imp_simple = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_simple.fit(X)
X_imp_simple = imp_simple.transform(X)

Iterative imputation with regression model:

In [ ]:
imp_it = IterativeImputer(max_iter=10, random_state=0)
imp_it.fit(X)
X_imp_it = imp_it.transform(X)

Creating two DataFrames with different imputation strategies:

In [ ]:
data_all_simple = data_all.copy()
data_all_it = data_all.copy()

for i in range (0, len(np.where(np.isnan(X))[0])):
    x = np.where(np.isnan(X))[0][i]
    y = np.where(np.isnan(X))[1][i]
    data_all_simple.iloc[x, y] = X_imp_simple[x, y]
    data_all_it.iloc[x, y] = X_imp_it[x, y]

After data preparation, final list of the features to select from:

In [ ]:
features = data_all.columns

len(features)

## Feature selection

Data split into train and test sets (stratifid by outcome):

In [ ]:
y_train, y_test = train_test_split(outcome_all, test_size=0.33, random_state=42, stratify=outcome_all)

data_train_simple = data_all_simple.loc[y_train.index]
data_test_simple = data_all_simple.loc[y_test.index]

data_train_it = data_all_it.loc[y_train.index]
data_test_it = data_all_it.loc[y_test.index]

Removing highly correlated features is a controversial step aimed to reduce the dimensionality of the feature space. Highly correlated features needlessly inflate the dimensionality of feature space. The idea is that highly correlated features can be grouped together and represented by one representative feature.  For features pairs with a high Spearman correlation (r > 0.9) the feature with the highest mean correlation with all remaining features is removed.

In [ ]:
def selectNonIntercorrelated(df_in, ftrs, corr_th):
    
    # selection of the features, which are not 'highly intercorrelated' (correlation is defined by Spearman coefficient);
    # pairwise correlation between all the features is calculated, 
    # from each pair of features, considered as intercorrelated, 
    # feature with maximum sum of all the pairwise Spearman correlation coefficients is a 'candidate to be dropped'
    # for stability of the selected features, bootstrapping approach is used: 
    # in each bootstrap split, the random subsample, stratified in relation to outcome, 
    # is formed, based on original observations from input dataset;
    # in each bootstrap split, 'candidates to be dropped' are detected;
    # for each input feature, its frequency to appear as 'candidate to be dropped' is calculated,
    # features, appeared in 50 % of splits as 'candidate to be dropped', are excluded from feature set
    
    # input:
    # df_in - input dataframe, containing feature values (dataframe, columns = features, rows = observations),
    # ftrs - list of dataframe features, used in analysis (list of feature names - string variables),
    # corr_th - threshold for Spearman correlation coefficient, defining each pair of features as intercorrelated (float)
    
    # output:
    # non_intercorrelated_features - list of names of features, which did not appear as inter-correlated
    
    corr_matrix = df_in.corr(method='spearman').abs()
    mean_absolute_corr = corr_matrix.mean()
    intercorrelated_features_set = []
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    high_corrs = upper.where(upper > corr_th).dropna(how='all', axis=1).dropna(how='all', axis=0)

    for feature in high_corrs.columns:
        mean_absolute_main = mean_absolute_corr[feature]
        correlated_with_feature = high_corrs[feature].index[pd.notnull(high_corrs[feature])]
        for each_correlated_feature in correlated_with_feature:
            mean_absolute = mean_absolute_corr[each_correlated_feature]
            if mean_absolute_main > mean_absolute:
                if feature not in intercorrelated_features_set:
                    intercorrelated_features_set.append(feature)
            else:
                if each_correlated_feature not in intercorrelated_features_set:
                    intercorrelated_features_set.append(each_correlated_feature)

    non_intercorrelated_features_set = [e for e in ftrs if e not in intercorrelated_features_set] 
    
    print ('Non intercorrelated features: ', non_intercorrelated_features_set)
    
    return non_intercorrelated_features_set

Selecting non-inter-correlated features:

In [ ]:
features_simple_non_intercorrelated = selectNonIntercorrelated(data_train_simple, features, 0.9)
features_it_non_intercorrelated = selectNonIntercorrelated(data_train_it, features, 0.9)
print ('Number of non-intercorrelated features (simple, iterative): ', 
       len(features_simple_non_intercorrelated), 
       len(features_it_non_intercorrelated))

There are some rules of thumb on how many features we need in the end:  
* $int(\frac{N_{samples}}{10})$ (Abu-Mostafa, Y. S., Magdon-Ismail, M., & Lin, H. T. (2012). Learning from data (Vol. 4, p. 4). New York: AMLBook.)  
* $\sqrt{N_{samples}}$ (Hua, J., Xiong, Z., Lowey, J., Suh, E., & Dougherty, E. R. (2005). Optimal number of features as a function of sample size for various classification rules. Bioinformatics, 21(8), 1509-1515.)

In [ ]:
print ('Nmber of samples in training dataset: ', len(y_train))
print ('Number of features to select according to Abu-Mostafa: ', int(len(y_train)/10))
print ('Number of features to select according to Hua: ', int(len(y_train)**0.5))

Below we implement Recursive Feature Elimination, RFE (https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html), based on Random Forest Classifier, RFC (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

In [ ]:
def selectRFE(df_in, ftrs, outcome, n_to_select):
    
    estimator = RandomForestClassifier(n_estimators=100, random_state=np.random.seed(0))
    selector = RFE(estimator, n_features_to_select=n_to_select, step=1)
    selector = selector.fit(df_in[ftrs], outcome)
    support = selector.get_support()
    selected_features_set = df_in[ftrs].loc[:, support].columns.tolist()
    
    return selected_features_set

Final feature sets for both imputation methods:

In [ ]:
features_simple_rfe = selectRFE(data_train_simple, features_simple_non_intercorrelated, y_train, 9)
features_it_rfe = selectRFE(data_train_it, features_it_non_intercorrelated, y_train, 9)

print (features_simple_rfe)
print (features_it_rfe)

## Modeling

### Model 1: Random Forest Classifier  
Training on train set, calculating outcomes for test set.

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=np.random.seed(0))
rfc.fit(data_train_simple[features_simple_rfe], y_train)
y_pred_rfc = rfc.predict(data_test_simple[features_simple_rfe])

Some classification metrics:

In [ ]:
print (classification_report(y_test, y_pred_rfc))

Typical classification metric ROC AUC score:

In [ ]:
fpr, tpr, _ = roc_curve(y_test, rfc.predict_proba(data_test_simple[features_simple_rfe])[:, 1], pos_label='1')
roc_auc = roc_auc_score(y_test, rfc.predict_proba(data_test_simple[features_simple_rfe])[:, 1])

plt.plot(fpr, tpr)
plt.title('RFC ROC curve (AUC = {})'.format(roc_auc))
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

Confusion matrix:

In [ ]:
cm = confusion_matrix(y_test, y_pred_rfc)
f = sns.heatmap(cm, annot=True)
plt.title('Confusion matrix for RFC')
plt.show()

### Model 2: Logistic Regression Classifier
Training and calculating labels for test set.

In [ ]:
lrc = LogisticRegression(random_state=0).fit(data_train_simple[features_simple_rfe], y_train)
y_pred_lrc = lrc.predict(data_test_simple[features_simple_rfe])

Classification metrics

In [ ]:
print (classification_report(y_test, y_pred_lrc))

ROC + ROC AUC:

In [ ]:
fpr, tpr, _ = roc_curve(y_test, lrc.predict_proba(data_test_simple[features_simple_rfe])[:, 1], pos_label='1')
roc_auc = roc_auc_score(y_test, lrc.predict_proba(data_test_simple[features_simple_rfe])[:, 1])

plt.plot(fpr, tpr)
plt.title('LRC ROC curve (AUC = {})'.format(roc_auc))
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

Confusion matrix:

In [ ]:
cm = confusion_matrix(y_test, y_pred_lrc)
f = sns.heatmap(cm, annot=True)
plt.title('Confusion matrix for LRC')
plt.show()

The same modeling steps you can perform for iterative imputed data to compare the imputation methods.